In [1]:
import numpy as np
import keras
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
IMG_SIZE = (224, 224)  # размер входного изображения сети

In [3]:
from keras.applications.inception_resnet_v2 import *

In [38]:
import re
from random import shuffle
from glob import glob

train_files = glob('../../../Data_cats/train/*.jpg')
test_files = glob('../../../Data_cats/test/*.jpg')

# загружаем входное изображение и предобрабатываем
def load_image(path, target_size=IMG_SIZE):
    img = load_img(path, target_size=target_size)  # загрузка и масштабирование изображения
    array = img_to_array(img)
    return preprocess_input(array)  # предобработка для VGG16

# генератор для последовательного чтения обучающих данных с диска
def fit_generator(files, batch_size=32):
    while True:
        shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)
            x = np.array([load_image(path) for path in files[i:j]])
            y = np.array([1. if re.match('.*dog\.\d', path) else 0. for path in files[i:j]])
            yield (x, y)

# генератор последовательного чтения тестовых данных с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

In [7]:
shuffle(train_files)  # перемешиваем обучающую выборку

train_val_split = 100

In [8]:
next(fit_generator(train_files[:train_val_split], train_val_split))[1]

array([0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
       1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.])

In [9]:
base_model = InceptionResNetV2(include_top = False,
                   weights = 'imagenet',pooling='max',
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

In [10]:
# фиксируем все веса предобученной сети
for layer in base_model.layers:
    layer.trainable = False

In [11]:
x = base_model.layers[-1].output
x = keras.layers.Dense(100,  # один выход
                activation='tanh',  # функция активации  
                kernel_regularizer=keras.regularizers.l1(1e-4))(x)
x = keras.layers.Dense(1,  # один выход
                activation='sigmoid',  # функция активации  
                kernel_regularizer=keras.regularizers.l1(1e-4))(x)
model = Model(inputs=base_model.input, outputs=x)

In [12]:
from keras.optimizers import Adam, SGD

mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(optimizer=mypotim, 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss
              metrics=['accuracy'])

BS = 32
EP = 2

In [13]:
a = shuffle(train_files)

In [14]:
a

In [15]:
shuffle(train_files)  # перемешиваем обучающую выборку

train_val_split = 100  # число изображений в валидационной выборке

validation_data = next(fit_generator(train_files[:train_val_split], train_val_split))

# запускаем процесс обучения
model.fit_generator(fit_generator(train_files[train_val_split:], batch_size = BS),  # данные читаем функцией-генератором
        steps_per_epoch=len(train_files)/BS,  # число вызовов генератора за эпоху
        epochs=EP,
        shuffle=True,# число эпох обучения
        validation_data=validation_data)

Epoch 1/2
782/781 [==============================] - 332s 425ms/step - loss: 0.4023 - acc: 0.9353 - val_loss: 0.2366 - val_acc: 0.9600
Epoch 2/2
782/781 [==============================] - 325s 415ms/step - loss: 0.2798 - acc: 0.9445 - val_loss: 0.1879 - val_acc: 0.9700


In [16]:
model.fit_generator(fit_generator(train_files[train_val_split:], batch_size = BS),  # данные читаем функцией-генератором
        steps_per_epoch=len(train_files)/BS,  # число вызовов генератора за эпоху
        epochs=10,
        shuffle=True,# число эпох обучения
        validation_data=validation_data)

Epoch 1/10
782/781 [==============================] - 325s 415ms/step - loss: 0.2700 - acc: 0.9427 - val_loss: 0.1584 - val_acc: 0.9600
Epoch 2/10
782/781 [==============================] - 325s 416ms/step - loss: 0.2487 - acc: 0.9448 - val_loss: 0.1419 - val_acc: 0.9800
Epoch 3/10
782/781 [==============================] - 325s 416ms/step - loss: 0.2400 - acc: 0.9504 - val_loss: 0.1144 - val_acc: 0.9900
Epoch 4/10
782/781 [==============================] - 325s 416ms/step - loss: 0.2356 - acc: 0.9518 - val_loss: 0.1469 - val_acc: 0.9600
Epoch 5/10
782/781 [==============================] - 325s 416ms/step - loss: 0.2425 - acc: 0.9471 - val_loss: 0.1208 - val_acc: 0.9900
Epoch 6/10
782/781 [==============================] - 325s 416ms/step - loss: 0.2469 - acc: 0.9473 - val_loss: 0.3271 - val_acc: 0.9400
Epoch 7/10
782/781 [==============================] - 325s 416ms/step - loss: 0.2602 - acc: 0.9460 - val_loss: 0.1720 - val_acc: 0.9900
Epoch 8/10
782/781 [============================

In [40]:
pred = model.predict_generator(predict_generator(test_files), len(test_files), max_queue_size=500)

In [41]:
with open('submit.txt', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, pred):
        dst.write('%s,%f\n' % (re.search('(\d+)', path).group(0), score))

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('submit.txt')

In [44]:
df.set_index('id').to_csv('sub.csv')